**Ad Channel Predictor Using NLP**
Objective:
The project predicts the best ad channel (Instagram, LinkedIn, Email, etc.) for a given ad text using Natural Language Processing (NLP).

Tools Used:
Python for coding.

Pandas for handling the data.

NLTK for cleaning and processing the text.

Scikit-learn for creating a machine learning model.

Gradio to create a simple web interface where users can input ad text and get predictions.

In [1]:
pip install -q pandas scikit-learn nltk


In [3]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/ad_channel_dataset.csv")

# Preprocess function
def preprocess(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in string.punctuation and t not in stopwords.words('english')]
    return ' '.join(tokens)

# Apply preprocessing
df['clean_text'] = df['ad_text'].apply(preprocess)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['channel']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict & Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Predict on new ad text
def predict_channel(ad_text):
    cleaned = preprocess(ad_text)
    vec = vectorizer.transform([cleaned])
    return model.predict(vec)[0]

# Example usage
sample = "Sign up for our free digital marketing webinar this Thursday!"
print("\nRecommended channel:", predict_channel(sample))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

       Email       1.00      1.00      1.00        36
    Facebook       1.00      1.00      1.00        35
  Google Ads       1.00      1.00      1.00        25
   Instagram       1.00      1.00      1.00        32
    LinkedIn       1.00      1.00      1.00        19
     Twitter       1.00      1.00      1.00        23

    accuracy                           1.00       170
   macro avg       1.00      1.00      1.00       170
weighted avg       1.00      1.00      1.00       170


Confusion Matrix:
 [[36  0  0  0  0  0]
 [ 0 35  0  0  0  0]
 [ 0  0 25  0  0  0]
 [ 0  0  0 32  0  0]
 [ 0  0  0  0 19  0]
 [ 0  0  0  0  0 23]]

Recommended channel: Google Ads


In [4]:
# --- Predicting from user input ---
while True:
    print("\nEnter ad content (or type 'exit' to quit):")
    user_input = input("> ")
    if user_input.lower() == 'exit':
        break
    predicted_channel = predict_channel(user_input)
    print("✅ Recommended Channel:", predicted_channel)



Enter ad content (or type 'exit' to quit):
> sneakers on sale
✅ Recommended Channel: Google Ads

Enter ad content (or type 'exit' to quit):
> AI course
✅ Recommended Channel: LinkedIn

Enter ad content (or type 'exit' to quit):
> exit


In [6]:
pip install -q gradio


In [7]:
import gradio as gr

# Gradio prediction function
def gradio_predict(ad_text):
    cleaned = preprocess(ad_text)
    vec = vectorizer.transform([cleaned])
    prediction = model.predict(vec)[0]
    return f"✅ Recommended Ad Channel: {prediction}"

# Gradio interface
demo = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Textbox(lines=4, placeholder="Enter your ad content here..."),
    outputs="text",
    title="Ad Channel Predictor",
    description="Enter ad text to predict the best advertising channel (e.g., Instagram, LinkedIn, Email).",
    theme="default"
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a50dd768c29016dfe6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
